In [1]:
import os, random
import cv2
import math
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm


In [2]:
meta_data = pd.read_csv(f"inputs\\train_metadata.csv")
meta_data.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg


In [3]:
meta_data = meta_data[["primary_label", "filename"]]

label_list = sorted(meta_data["primary_label"].unique())
label_id_list = list(range(len(label_list)))
label2id = dict(zip(label_list, label_id_list))
id2label = dict(zip(label_id_list, label_list))

In [24]:
def get_Ebird_list (df_path):
    '''
    Return set of EBird code in input annotation df dataset
    '''
    df_CCR = pd.read_csv(df_path)
    eBird = df_CCR["Species eBird Code"].unique()
    print(f"{df_path.split('/')[-2]} got {len(eBird)} eBird")
    return eBird

In [25]:
list_df_path = [
    "inputs/previous_dataset/ColumbiaAndCostaRica/annotations.csv",
    "inputs/previous_dataset/Hawaii/annotations.csv",
    "inputs/previous_dataset/SouthwesternAmazonBasin/annotations.csv",
    "inputs/previous_dataset/WesternUS/annotations.csv"
]

eBird_CCR = get_Ebird_list(list_df_path[0])
eBird_H = get_Ebird_list(list_df_path[1])
eBird_SAB = get_Ebird_list(list_df_path[2])
eBird_WUS = get_Ebird_list(list_df_path[3])


ColumbiaAndCostaRica got 90 eBird
Hawaii got 27 eBird
SouthwesternAmazonBasin got 133 eBird
WesternUS got 56 eBird


In [32]:
print(list(set(eBird_CCR) & set(eBird_H) & set(eBird_SAB) & set(eBird_WUS)))
print(list(set(eBird_WUS) & set(label_list)))

[]
[]


In [36]:
label_list

['asbfly',
 'ashdro1',
 'ashpri1',
 'ashwoo2',
 'asikoe2',
 'asiope1',
 'aspfly1',
 'aspswi1',
 'barfly1',
 'barswa',
 'bcnher',
 'bkcbul1',
 'bkrfla1',
 'bkskit1',
 'bkwsti',
 'bladro1',
 'blaeag1',
 'blakit1',
 'blhori1',
 'blnmon1',
 'blrwar1',
 'bncwoo3',
 'brakit1',
 'brasta1',
 'brcful1',
 'brfowl1',
 'brnhao1',
 'brnshr',
 'brodro1',
 'brwjac1',
 'brwowl1',
 'btbeat1',
 'bwfshr1',
 'categr',
 'chbeat1',
 'cohcuc1',
 'comfla1',
 'comgre',
 'comior1',
 'comkin1',
 'commoo3',
 'commyn',
 'compea',
 'comros',
 'comsan',
 'comtai1',
 'copbar1',
 'crbsun2',
 'cregos1',
 'crfbar1',
 'crseag1',
 'dafbab1',
 'darter2',
 'eaywag1',
 'emedov2',
 'eucdov',
 'eurbla2',
 'eurcoo',
 'forwag1',
 'gargan',
 'gloibi',
 'goflea1',
 'graher1',
 'grbeat1',
 'grecou1',
 'greegr',
 'grefla1',
 'grehor1',
 'grejun2',
 'grenig1',
 'grewar3',
 'grnsan',
 'grnwar1',
 'grtdro1',
 'gryfra',
 'grynig2',
 'grywag',
 'gybpri1',
 'gyhcaf1',
 'heswoo1',
 'hoopoe',
 'houcro1',
 'houspa',
 'inbrob1',
 'indpit1',
 